# Importing Libraries
performing all necessary imports here

In [1]:
# import model related libraries
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# import huggingface login
from huggingface_hub import login

# import the accesstoken of huggingface, which is saved in .env file
# from dotenv import load_dotenv # it was not running on kaggle so i have commented it
import os

# import time to compute how long it took the model to run
import time

# import date-time to display the date and time of last run
from datetime import datetime

# import text wrap to make sure its fully displayable
import textwrap

# Initialize Inputs
here we initailize all inputs given, like the input text, the prompt, and the model name

### Input Text
here is the email we will be passing for all models: this is given in the question and stays constant

In [2]:
input_text = """Subject: Concerns About Professor X’s Conduct 
 
Dear Dr. Ustaad, 
I hope this email finds you well. I am writing to express my concerns about Professor X’s conduct during the Introduction to Zoology class last semester. On multiple occasions, Professor X made dismissive remarks about students’ questions and failed to provide clear feedback on assignments.  
 
Additionally, the grading seemed inconsistent and unfair, with no opportunity for clarification or appeal. 
 
I found this experience deeply frustrating and demotivating, and I believe it affected my performance in the course. I would appreciate it if the department could look into this matter and ensure that future students have a more positive and supportive learning environment. 
 
Thank you for your attention to this matter. 
Sincerely, 
Shaagird
"""

print("Input Text:\n", input_text)

Input Text:
 Subject: Concerns About Professor X’s Conduct 
 
Dear Dr. Ustaad, 
I hope this email finds you well. I am writing to express my concerns about Professor X’s conduct during the Introduction to Zoology class last semester. On multiple occasions, Professor X made dismissive remarks about students’ questions and failed to provide clear feedback on assignments.  
 
Additionally, the grading seemed inconsistent and unfair, with no opportunity for clarification or appeal. 
 
I found this experience deeply frustrating and demotivating, and I believe it affected my performance in the course. I would appreciate it if the department could look into this matter and ensure that future students have a more positive and supportive learning environment. 
 
Thank you for your attention to this matter. 
Sincerely, 
Shaagird



### Model Params
here we intialize the model name, its parameters, and then the text_generator we will be using

In [3]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
# doesnt work on huggingface: "mistralai/Mistral-3B-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [4]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=5000,
    do_sample=False
)

Device set to use cuda:0


# Login, Authentication
here we authenticate the hugging face access token

In [5]:
# first we load the access token from the .env hidden file (it is a gitignore file)
# load_dotenv() # it was not running on kaggle so i have commentated it
# api_token = os.getenv("HUGGINGFACE_API_TOKEN")
api_token = "" # put your api token here, have removed for github security

In [6]:
login(api_token)

# Model Running
here we run the model, but before that we display the model details and also compute the timetaken to run the model

### Model Details
here we display how many parameters and how much memory the model took

In [7]:
print(model.dtype)

# Parameters Computation
total_params = sum(p.numel() for p in model.parameters())
print(f"Model Total Parameters: {total_params / 1e9:.2f} billion")

# Memory used Computation (in MBs)
memory = total_params * 2 / (1024 ** 2)  
print(f"Estimate Memory Footprint: {memory:.2f} MB\n")

torch.bfloat16
Model Total Parameters: 3.09 billion
Estimate Memory Footprint: 5885.96 MB



## Model Running
here for every task, we will first save the start_time and the end_time and then compute the timetaken

In [8]:
def compute_time(start_time, end_time):
    """This function calculates the time difference between two given times in seconds. For this given problem, this function computes the timetaken by a model to run and perform a task

    Args:
        start_time (time): start time of the model
        end_time (time): end time of the model

    Returns:
        string: the time taken in hours, minutes, seconds, and mili-seconds
    """
    elapsed_time = end_time - start_time
    hours, rem = divmod(elapsed_time, 3600)
    minutes, rem = divmod(rem, 60)
    seconds, milliseconds = divmod(rem, 1)
    milliseconds *= 1000  # Convert seconds fraction to milliseconds
    
    return f"Time taken: {int(hours)}h {int(minutes)}m {int(seconds)}s {int(milliseconds)}ms"

### Prompt Making
here we define the prompt we will use for all tasks of summarization, question answering, keyword extraction and translation

In [9]:
prompt = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks.
2 - Answer “What was the reason for the student's disappointment with Professor X? ” based on the email content.
3 - Identify key details such as incidents, concerns, and requested actions. 
4 - Translate the following text into French.

Separate your answers with line breaks.
Text:
```{input_text}```
"""

### Model Execution
here we run the model by passing the prompt to a text_generator

In [10]:
# start the timer
start_time = time.time()

# generate the output
messages = [
    {"role": "user", "content": prompt}
]
output = generator(messages)

# end the timer
end_time = time.time()

# display time taken with output
print(compute_time(start_time, end_time))
print("Output:\n", output[0]["generated_text"])#textwrap.fill(output[0]["generated_text"], width=80))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Time taken: 0h 0m 25s 20ms
Output:
 ```markdown
Summary:
The student expresses disappointment with Professor X due to dismissive remarks, inconsistent grading, and lack of feedback and appeal opportunities during the Introduction to Zoology class. The student believes these issues negatively impacted their performance and hopes the department will address them to improve the learning environment for future students.

Answer:
The reason for the student's disappointment with Professor X was the dismissive remarks made towards students' questions, inconsistent and unfair grading, and the absence of opportunities for clarification or appeal.

Key Details:
- Multiple instances of dismissive remarks from Professor X regarding students' questions.
- Inconsistent and unfair grading practices.
- Lack of opportunities for students to clarify or appeal grades.

Translation:
```français
Texte traduit en français :
Objet : Préoccupations concernant le comportement de M. X

Cher Dr. Ustaad,
J'espère

In [14]:
prompt_back_to_eng = """
Translate the following text to English:
Texte traduit en français :
Objet : Préoccupations concernant le comportement de M. X

Cher Dr. Ustaad,
J'espère que cette lettre vous trouve bien. Je vous écris pour exprimer mes préoccupations concernant le comportement de M. X lors de la classe d'Introduction à la zoologie l'an dernier. De nombreuses fois, M. X a fait des remarques moqueuses sur les questions des étudiants et n'a pas fourni une feedback claire sur les devoirs.
De plus, la notation semblait inégale et injuste, sans possibilité de clarification ou de recours.
Je trouve cette expérience profondément frustrante et démotivante, et je crois qu'elle a affecté ma performance dans le cours. J'apprécierais que la département examine ce sujet et assure un environnement de formation plus positif et soutenu pour les futurs étudiants.
Merci de votre attention à ce sujet.
Sincèrement,
Shaagird
"""

In [15]:
# start the timer
start_time = time.time()

# generate the output
messages = [
    {"role": "user", "content": prompt_back_to_eng}
]
output = generator(messages)

# end the timer
end_time = time.time()

# display time taken with output
print(compute_time(start_time, end_time))
print("Output:\n", output[0]["generated_text"])#textwrap.fill(output[0]["generated_text"], width=80))

Time taken: 0h 0m 10s 73ms
Output:
 Subject: Concerning Mr. X's Behavior

Dear Dr. Ustaad,

I hope this letter finds you well. I am writing to express my concerns regarding Mr. X's behavior during the Introduction to Zoology class last year. Mr. X made mocking remarks about students' questions and did not provide clear feedback on assignments.

Moreover, the grading seemed unfair and unequal, with no possibility for clarification or appeal.

I find this experience deeply frustrating and demotivating, and I believe it has affected my performance in the course. I would appreciate it if the department examines this matter and ensures a more positive and supportive learning environment for future students.

Thank you for your attention to this issue.

Sincerely,
Shaagird


# End
here we display the last time this jupyter notebook was run to always remember it

In [16]:
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2025-02-02 11:03:50
